In [275]:
import pandas as pd
import datetime

In [276]:
midi = pd.read_csv('merged_midi.csv')

In [277]:
#PLAN

# 1. Convert to rows
# 2. Add US / RU / Other intervention dummies
# Maak mergebaar

In [278]:
# df['start'] = pd.to_datetime(df['start'][:5])
midi['start'] = midi['start'].astype(str).str[:4]
midi['start'] = midi['start'].astype(int)

# Remove error codes from datefields
mask = midi['start'] < 1992
midi = midi[mask]
mask = midi['start'] > 1945
midi = midi[mask]

midi['end'] = midi['end'].astype(str).str[:4]
midi['end'] = midi['end'].astype(int)

# Remove error codes from datefields
mask = midi['end'] < 2020
midi = midi[mask]
mask = midi['end'] > 1945
midi = midi[mask]

In [279]:
midi = midi[['intervener', 'target', 'start', 'end']]

In [280]:
midi['duration'] = midi['end'] - midi['start'] + 1
# Remove error-code enddates
mask = midi['duration'] < 5000
midi = midi[mask]
midi_backup = midi
midi.columns

Index(['intervener', 'target', 'start', 'end', 'duration'], dtype='object')

In [281]:
midi.start = pd.to_datetime(midi.start, format='%Y')
midi.end = pd.to_datetime(midi.end, format='%Y')
midi.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 767 entries, 0 to 779
Data columns (total 5 columns):
intervener    767 non-null int64
target        767 non-null int64
start         767 non-null datetime64[ns]
end           767 non-null datetime64[ns]
duration      767 non-null int64
dtypes: datetime64[ns](2), int64(3)
memory usage: 36.0 KB


In [282]:
midi_years = pd.concat([pd.DataFrame({'date': pd.date_range(row.start, row.end + pd.DateOffset(years=1), freq='12m'), 
              'target': row.target, 'intervener': row.intervener}, columns=['date', 'target', 'intervener']) for i, row in midi.iterrows()])

midi_years
# dummies = midi_years['intervener'].astype('str').str.get_dummies()
# midi_years = pd.concat([midi_years, dummies], axis=1)
# us_mask = midi_years['target'] == 200
# us = midi_years[us_mask]
# us

midi = midi_years

# remove dates after cold war
mask = midi['date'] < datetime.date(1992,1,1)
midi = midi[mask]
# Reconvert date to year
midi['date'] = midi['date'].dt.year

# Remove all interventions we dont care about
nations = [2, 200, 210, 211, 220, 230, 235, 365]
mask = midi['intervener'].isin(nations)
midi = midi[mask]
dummies = midi['intervener'].astype('str').str.get_dummies()
midi = pd.concat([midi, dummies], axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [283]:
# Group dataframe, so one year-target pair exists for each country


In [284]:
midi = midi.groupby(['target', 'date']).max()

In [287]:
midi

intervener  1  100  101  1014  130  135  160  2  20 ...  850  90  \
target date                                                      ...            
31     1980           2  0    0    0     0    0    0    0  1   0 ...    0   0   
40     1958           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1959           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1960           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1962         365  0    0    0     0    0    0    0  0   0 ...    0   0   
41     1963           2  0    0    0     0    0    0    0  1   0 ...    0   0   
42     1961           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1965           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1966           2  0    0    0     0    0    0    0  1   0 ...    0   0   
55     1983           2  0    0    0     0    0    0    0  1   0 ...    0   0   
61     1969         200  0    0    0     0    0    0    0  0   0 ...    0   0   
90     1987           2  0    0    0     0    0    0    0  1   0 ...    0   0   
91     1986           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1987           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1988           2  0    0    0     0    0    0    0  1   0 ...    0   0   
92     1983           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1984           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1985           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1986           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1987           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1988           2  0    0    0     0    0    0    0  1   0 ...    0   0   
93     1979           2  0    0    0     0    0    0    0  1   0 ...    0   0   
94     1979           2  0    0    0     0    0    0    0  1   0 ...    0   0   
95     1959           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1988           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1989           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1990           2  0    0    0     0    0    0    0  1   0 ...    0   0   
100    1948           2  0    0    0     0    0    0    0  1   0 ...    0   0   
110    1978           2  0    0    0     0    0    0    0  1   0 ...    0   0   
145    1979           2  0    0    0     0    0    0    0  1   0 ...    0   0   
...                 ... ..  ...  ...   ...  ...  ...  ... ..  .. ...  ...  ..   
816    1969           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1970           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1971           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1972           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1973           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1974           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1975           2  0    0    0     0    0    0    0  1   0 ...    0   0   
817    1961           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1962           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1963           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1964           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1965           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1966           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1967           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1968           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1969           2  0    0    0     0    0    0    0  1   0 ...    0   0   
       1970           2  0    0    0     0  

In [285]:
midi.to_csv('delhierna.csv')

In [286]:
midi.duplicated(['date', 'target']).value_counts()

ValueError: not enough values to unpack (expected 2, got 0)